### Projeto 1 - Ciência dos Dados

Nome: __João Lucas Brasileiro de Sousa Lopes__

Nome: __Láisa Camilly de Oliveira__

Nome: __Nathan Braga__

___
Carregando algumas bibliotecas:

In [84]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import unidecode

In [85]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\laisa\OneDrive\Área de Trabalho\Insper - Eng. Comp\2º semestre\C.Dados\Projeto 1\Projeto-1


Carregando a base de dados com as mensagens dos seus arquivos:

___
#### Classificador automático.
- O objetivo deste projeto é desenvolver um classificador baseado no algoritmo Naive Bayes, capaz de determinar se uma determinada frase pertence à área de Química ou Física.
- O dataset utilizado consiste em frases de ambas as disciplinas, associadas a rótulos que identificam a qual matéria elas pertencem.

___
#### Montando um Classificador Naive-Bayes.
<p>Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.</p>

- Criação de uma função responsável pela limpeza do dataset analisado, incluindo stopwords, acentos, e caracteres.

In [86]:
def limpeza(texto):

    """
    Limpa o texto fornecido, tirando caracteres indesejados.
    """

    caracteres = '[´"!-.:?;\$\n'']'
    limpar = re.compile(caracteres)
    texto_limpo = re.sub(limpar, '', texto)

    texto_limpo = unidecode.unidecode(texto_limpo)
    return texto_limpo

In [87]:
# Criando nossa base de dados para treinamento e teste:
train = pd.read_csv('dados_treino_TRIO_joaolbsl.csv')

test = pd.read_csv('dados_teste_TRIO_joaolbsl.csv')

In [88]:
# Chamando a função criada para tratar a base de dados Train:
for i in range(0,train.shape[0]):
    train['Comment'][i] = limpeza(train['Comment'][i])
    train['Comment'][i] = train['Comment'][i].lower().strip()

In [89]:
# Separando o df para o topico Física e Química.
train_fis = train.loc[train['Topic'] == 'Physics', 'Comment'] 
train_qui = train.loc[train['Topic'] == 'Chemistry', 'Comment'] 

In [90]:
# Lista com todas as palavras que aparecem no df de física.
palavras_fis = []

for frase in train_fis:
    frase = frase.replace('\\n', ' ')
    frase = frase.split()

    for word in frase:
        palavras_fis.append(word)

# Lista com todas as palavras que aparecem no df de química.
palavras_qui = []

for frase in train_qui:
    frase = frase.replace('\\n', ' ')
    frase = frase.split()

    for word in frase:
        palavras_qui.append(word)


In [91]:
# Lista com palavras de física e química juntas.
todas_palavras = (palavras_fis + palavras_qui)

- Criação da função que calcula a probabilidade de tal palavra ser de tal tópico.

In [92]:
def calcula_prob (frase, palavras_topico, total_palavras_topico, todas_palavras):

    """
    Calcula a probabilidadea de uma palavra do topico de física ou química estar na frase.
    """

    prob = 1
    frase = frase.split()
    for palavra in frase:
        if palavra in todas_palavras:
            prob = prob * (palavras_topico.count(palavra) + 1) / (total_palavras_topico + len(todas_palavras))

    return prob

In [93]:
# Quantidade de palavras em Física e Química, para chamar na minha função.
total_palavras_fis = len(palavras_fis)
total_palavras_qui= len(palavras_qui)

In [94]:
# # Testando a função criada baseado no classificador, dado uma frase específica:
frase = "the physics of the universe"
prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

# Vendo o valor:
print(f"Probabilidade de ser Física: {prob_fis}")
print(f"Probabilidade de ser Química: {prob_qui}")

Probabilidade de ser Física: 6.830019490911401e-13
Probabilidade de ser Química: 2.5488276519468887e-15


In [95]:
# Todas as frases do df Treino:
frases_train = train['Comment']

# Todas as frases do df Teste:
frases_test = test['Comment']

___
#### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Treinamento.

In [57]:
# Fazendo com que nosso robô "Boot", calcule as probabilidades do df Train.
topico_esperado = []

for frase in frases_train:

    prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
    prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

    if prob_fis > prob_qui:
        topico_esperado.append("Physics")
    else:
        topico_esperado.append("Chemistry")

train['Boot'] = topico_esperado

tabela_frequencia_relativa_train = pd.crosstab(train['Topic'], train['Boot'], normalize = True, margins =True)

print(tabela_frequencia_relativa_train)

Boot       Chemistry   Physics       All
Topic                                   
Chemistry   0.515143  0.040571  0.555714
Physics     0.089429  0.354857  0.444286
All         0.604571  0.395429  1.000000


In [64]:
# Verdadeiros Positivos.
vp_train = tabela_frequencia_relativa_train['Chemistry']['Chemistry'] / tabela_frequencia_relativa_train['All']['Chemistry']
print(f'Vp:{vp_train}')

Vp:0.9269922879177378


In [65]:
# Falsos Positivos.
fp_train = tabela_frequencia_relativa_train['Chemistry']['Physics']/tabela_frequencia_relativa_train['All']['Physics']
print(f'Fp:{fp_train}')

Fp:0.2012861736334405


In [73]:
# Verdadeiros Negativos.
vn_train = 1 - fp_train
print(f'Vn: {vn_train}')

Vn: 0.7987138263665595


In [81]:
# Acurácia de Treino.
acuracia_train = (tabela_frequencia_relativa_train['Chemistry']['Chemistry'] + tabela_frequencia_relativa_train['Physics']['Physics'])*100
print(f'Acurácia df.Train: {acuracia_train:.2f}')

Acurácia df.Train: 87.00


___
#### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Teste.

In [60]:
# Fazendo com que nosso robô "Boot", calcule as probabilidades do df Test.
topico_esperado = []

for frase in frases_test:

    prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
    prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

    if prob_fis > prob_qui:
        topico_esperado.append("Physics")
    else:
        topico_esperado.append("Chemistry")

test['Boot'] = topico_esperado

tabela_frequencia_relativa_test = pd.crosstab(test['Topic'], test['Boot'], normalize = True, margins = True)

print(tabela_frequencia_relativa_test)

Boot       Chemistry   Physics       All
Topic                                   
Chemistry   0.443333  0.131333  0.574667
Physics     0.130000  0.295333  0.425333
All         0.573333  0.426667  1.000000


In [68]:
# Verdadeiros Positivos.
vp_test = tabela_frequencia_relativa_test['Chemistry']['Chemistry'] / tabela_frequencia_relativa_test['All']['Chemistry']
print(f'Vp: {vp_test}')

Vp: 0.771461716937355


In [71]:
# Falsos Positivos. 
fp_test = tabela_frequencia_relativa_test['Chemistry']['Physics']/tabela_frequencia_relativa_test['All']['Physics']
print(f'Fp: {fp_test}')

Fp: 0.3056426332288401


In [74]:
# Verdadeiros Negativos.
vn_test = 1 - fp_test
print(f'Vn: {vn_test}')

Vn: 0.6943573667711599


In [80]:
# Acurácia do df.Test.
acuracia_test = (tabela_frequencia_relativa_test['Chemistry']['Chemistry'] + tabela_frequencia_relativa_test['Physics']['Physics'])*100
print(f'Acurácia df.Test: {acuracia_test:.2f}')

Acurácia df.Test: 73.87


___
- Verificando o comportamento do nosso robô da base de treino e da base de teste, e calculando a diferenç percentual entre os dois.

In [83]:
# Diferença de acurácias:
diferenca  = acuracia_train - acuracia_test
print(f'Diferenças do percentual de acurácia entre Train e Test: {diferenca:.2f}')

Diferenças do percentual de acurácia entre Train e Test: 13.13


___
#### Conclusão.

- Considerando a base de Testes, a qual foi analisada, é possível dizer que o classificador é preciso. Tendo em vista que o classificador obteve 73.87% de acurácia.
- 
- Para verificar a performance do classificador foram calculadas as acurácias utilizando as duas bases de dados, a base de treino e a base de testes, sendo a primeira aproximadamente 2.3 vezes maior que a segunda.
- A acurácia utilizando a base de treino foi de 86.9% e com a base de teste foi de 74.13%, sendo a diferença entre as duas igual a 12.83%.
- Para o aperfeiçoamento do classificador, foi necessário a implementação da limpeza das duas bases de dados, removendo os caracteres indesejados e fazendo com que assim as bases de dados estejam mais limpas.

___
#### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
#### Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
#### Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**